In [1]:
import os
import datetime

import pandas as pd
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper', style='darkgrid', font_scale=1.2)

In [2]:
DATA_DIR = 'data'
CLEANED_FILE = 'clean_data_cro.csv' 
CLEANED_DATA_PATH = os.path.join(os.pardir, DATA_DIR, CLEANED_FILE)

df_w = pd.read_csv(CLEANED_DATA_PATH)

In [3]:
# assemble new dataframe for SEIR(D) modeling
columns_dtypes_dict = {
    'date': datetime.datetime,
    'total_tests': float,
    'confirmed_positives': float,
    'confirmed_deceased': float,
    'estimate_recovered': float,
}

# create empty dataframe with predifined data types
columns_dtypes_list = [(k, v) for k, v in columns_dtypes_dict.items()]
dtypes = np.dtype(columns_dtypes_list)
empty_matrix = np.empty(0, dtype=dtypes)
df_m = pd.DataFrame(empty_matrix)

In [4]:
df_m.date = df_w.testing_date.sort_values().unique()
df_m.date = pd.to_datetime(df_m.date)
df_m.total_tests = df_w.groupby('testing_date').count().testing_result.values
df_m.confirmed_positives = df_w.groupby('testing_date').sum().testing_result.values
df_m.confirmed_deceased = df_w.groupby('testing_date').sum().deceased.values

In [5]:
eff_start_date = datetime.datetime(2020, 2, 25)
eff_end_date_initial = datetime.datetime(2020, 6, 1)
df_m_initial = df_m.loc[df_m.date <= eff_end_date_initial]

In [6]:
df_m_initial

,date,total_tests,confirmed_positives,confirmed_deceased,estimate_recovered
0,2020-02-25,32,1.0,0.0,NaN
1,2020-02-26,41,1.0,0.0,NaN
2,2020-02-27,28,2.0,0.0,NaN
3,2020-02-28,13,2.0,0.0,NaN
4,2020-02-29,10,0.0,0.0,NaN
...,...,...,...,...,...
93,2020-05-28,456,1.0,0.0,NaN
94,2020-05-29,561,0.0,0.0,NaN
95,2020-05-30,258,0.0,0.0,NaN
96,2020-05-31,245,0.0,0.0,NaN
